# 基础因子加alpha191实时计算---中证800

中证800选股策略

In [1]:
%matplotlib inline
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../../../../')

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

from PyFin.api import *
from alphamind.api import *
from conf.models import *
from conf.config import*
from data.engines.model import Record
from alphamind.execution.naiveexecutor import NaiveExecutor
from stacking import factor_store, feature_list
from optimization.bayes_optimization_xgb import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)


In [2]:
# 指定需要执行的日期和保存的文件夹
# start_date = '2010-01-19'  # 训练集的起始时间
start_date = '2019-10-18'  # 训练集的起始时间
end_date = '2019-12-27'
# 设置保存的文件目录
weekly = '800st'  # s1: 周一, s2: 周二,  s3: 周三, s4: 周四, s5: 周五


In [3]:
universe = Universe('zz500')+ Universe('hs300')
freq = '5b'
benchmark_code = 906
ref_dates = makeSchedule(start_date, end_date, freq, 'china.sse')
horizon = map_freq(freq)
industry_name = 'sw'
industry_level = 1

# 前一个调仓日, 用于获取前一个调仓日的持仓信息
ref_date_pre = ref_dates[-2]
# 当前调仓日
ref_date = ref_dates[-1]

# 因子数据库
data_source = alpha_db
engine = SqlEngine(data_source)
ref_dates

[datetime.datetime(2019, 10, 18, 0, 0),
 datetime.datetime(2019, 10, 25, 0, 0),
 datetime.datetime(2019, 11, 1, 0, 0),
 datetime.datetime(2019, 11, 8, 0, 0),
 datetime.datetime(2019, 11, 15, 0, 0),
 datetime.datetime(2019, 11, 22, 0, 0),
 datetime.datetime(2019, 11, 29, 0, 0),
 datetime.datetime(2019, 12, 6, 0, 0),
 datetime.datetime(2019, 12, 13, 0, 0),
 datetime.datetime(2019, 12, 20, 0, 0),
 datetime.datetime(2019, 12, 27, 0, 0)]

In [4]:
# uqer因子列表
basic_factor_store = factor_store.basic_factor_store
# alpha191因子列表
alpha_factor_store = factor_store.alpha_factor_store

In [5]:
%%time
# 提取Uqer因子
basic_factor_org = engine.fetch_factor_range(universe, basic_factor_store, dates=ref_dates)
# 提取alpha191因子
# alpha191_factor_org = engine.fetch_factor_range(universe, 
#                                                 alpha_factor_store, 
#                                                 dates=ref_dates, 
#                                                 used_factor_tables=[Alpha191]).drop(['chgPct','secShortName'], axis=1)
# # 合并所有的因子
# factor_data_org = pd.merge(basic_factor_org, alpha191_factor_org, on=['trade_date', 'code'], how='outer')
factor_data_org = basic_factor_org


CPU times: user 12.9 s, sys: 529 ms, total: 13.4 s
Wall time: 15.4 s


In [6]:
set(factor_data_org['trade_date'])

{Timestamp('2019-10-18 00:00:00'),
 Timestamp('2019-10-25 00:00:00'),
 Timestamp('2019-11-01 00:00:00'),
 Timestamp('2019-11-08 00:00:00'),
 Timestamp('2019-11-15 00:00:00'),
 Timestamp('2019-11-22 00:00:00'),
 Timestamp('2019-11-29 00:00:00'),
 Timestamp('2019-12-06 00:00:00'),
 Timestamp('2019-12-13 00:00:00'),
 Timestamp('2019-12-20 00:00:00'),
 Timestamp('2019-12-27 00:00:00')}

In [7]:
len(factor_data_org)

8800

In [8]:
assert len(set(factor_data_org['trade_date'])) == len(ref_dates)


In [9]:
# 因子预处理
## 确失值填充
factor_mean = factor_data_org.mean()
factor_std = factor_data_org.std()
factor_data_org = factor_data_org.fillna(factor_mean)

In [10]:
%%time
# 获取行业数据
industry = engine.fetch_industry_range(universe, dates=ref_dates)
# factor_data = pd.merge(factor_data_org, industry, on=['trade_date', 'code']).fillna(0.)
factor_data = pd.merge(factor_data_org, industry, on=['trade_date', 'code'])

# 获取风险因子
risk_total = engine.fetch_risk_model_range(universe, dates=ref_dates)[1]


CPU times: user 210 ms, sys: 11 ms, total: 221 ms
Wall time: 379 ms


In [11]:
%%time
return_data = engine.fetch_dx_return_range(universe, dates=ref_dates, horizon=horizon, offset=0,benchmark = benchmark_code)


CPU times: user 182 ms, sys: 10 ms, total: 193 ms
Wall time: 534 ms


In [12]:
return_data.tail()

,trade_date,code,dx
7959,2019-12-20,603885,0.061350
7969,2019-12-20,603888,-0.035116
7979,2019-12-20,603939,-0.013378
7989,2019-12-20,603986,0.023002
7999,2019-12-20,603993,0.148576


In [13]:
%%time
benchmark_total = engine.fetch_benchmark_range(dates=ref_dates, benchmark=benchmark_code)
industry_total = engine.fetch_industry_matrix_range(universe, dates=ref_dates, category=industry_name, level=industry_level)

train_data = pd.merge(factor_data, return_data, on=['trade_date', 'code']).dropna()
len(train_data)


CPU times: user 137 ms, sys: 5.72 ms, total: 143 ms
Wall time: 202 ms


In [14]:
# 获取特征名
# uqer feature
features = list(basic_factor_store.keys())

# alpha features
# alpha_features = list(alpha_factor_store.keys())

# features = feature_list.uqer_features
# alpha_features = feature_list.alpha_features
# features.extend(alpha_features)

label = ['dx']

In [15]:
from datetime import datetime, timedelta
from models.m1_xgb import *
from conf.configuration import xgb_conf
from data.engines.model import Record
import xgboost as xgb
import gc

alpha_logger.info('{0} is start'.format(ref_date))

# machine learning model
## Filter Training data
## 训练集构造
trade_date_pre = ref_date - timedelta(days=1)
# trade_date_pre_80 = ref_date - timedelta(days=80)

## 1、选择调仓日当天之前(不含当天)并且在80天以内的因子数据作为训练集.
# train = train_data[(train_data.trade_date <= trade_date_pre) & (trade_date_pre_80 <= train_data.trade_date)].dropna()
## 2、选择调仓日当天之前(不含当天)的因子数据作为训练集.
train = train_data[train_data.trade_date <= trade_date_pre].dropna()
alpha_logger.info('trade_date_pre {0}'.format(trade_date_pre))

if len(train) <= 0:
    alpha_logger.info('{0} HAS NO TRAIN DATA!!!'.format(ref_date))

x_train = train[features]
y_train = train[label]
alpha_logger.info('len_x_train: {0}, len_y_train: {1}'.format(len(x_train.values), len(y_train.values)))
alpha_logger.info('X_train.shape={0}, X_test.shape = {1}'.format(np.shape(x_train), np.shape(y_train)))


2019-12-30 05:35:42,333 - ALPHA_MIND - INFO - 2019-12-27 00:00:00 is start
2019-12-30 05:35:42,393 - ALPHA_MIND - INFO - trade_date_pre 2019-12-26 00:00:00
2019-12-30 05:35:42,404 - ALPHA_MIND - INFO - len_x_train: 8000, len_y_train: 8000
2019-12-30 05:35:42,404 - ALPHA_MIND - INFO - X_train.shape=(8000, 423), X_test.shape = (8000, 1)


In [16]:
# load xgboost regression configuration
xgb_conf.xgb_config_r()
xgb_conf.cv_folds = None
xgb_conf.early_stop_round = 100
xgb_conf.max_round = 800
xgb_conf.params.update({'nthread': 12})

GPU_device = False
if GPU_device:
    # use GPUs
    xgb_conf.params.update({'tree_method': 'gpu_hist'})
alpha_logger.info("params before: {}".format(xgb_conf.params))
tic = time.time()

# hyper_parameters optimization
# opt_parameters = {'max_depth': (2, 12),
#                   'gamma': (0.001, 10.0),
#                   'min_child_weight': (0, 20),
#                   'max_delta_step': (0, 10),
#                   'subsample': (0.01, 0.99),
#                   'colsample_bytree': (0.01, 0.99)
#                  }

# opt_xgb = BayesOptimizationXGB('regression', x_train, y_train)
# params_op = opt_xgb.train_opt(opt_parameters)
# xgb_conf.params.update(params_op)
alpha_logger.info("params after: {}".format(xgb_conf.params))
alpha_logger.info("hyper params optimize time : {}".format(time.time() - tic))


2019-12-30 05:35:42,411 - ALPHA_MIND - INFO - params before: {'booster': 'gbtree', 'objective': 'reg:linear', 'eval_metric': ['rmse', 'logloss'], 'nthread': 12, 'silent': 1, 'learning_rate': 0.01, 'max_depth': 5, 'eta': 0.03, 'alpha': 0, 'lambda': 0, 'subsample': 0.7, 'colsample_bytree': 0.5, 'min_child_weight': 3, 'seed': 2019}
2019-12-30 05:35:42,412 - ALPHA_MIND - INFO - params after: {'booster': 'gbtree', 'objective': 'reg:linear', 'eval_metric': ['rmse', 'logloss'], 'nthread': 12, 'silent': 1, 'learning_rate': 0.01, 'max_depth': 5, 'eta': 0.03, 'alpha': 0, 'lambda': 0, 'subsample': 0.7, 'colsample_bytree': 0.5, 'min_child_weight': 3, 'seed': 2019}
2019-12-30 05:35:42,413 - ALPHA_MIND - INFO - hyper params optimize time : 0.0005588531494140625


In [17]:
# model training
xgb_model = XGBooster(xgb_conf)
alpha_logger.info('xgb_model params: \n{0}'.format(xgb_model.get_params()))

best_score, best_round, best_model = xgb_model.fit(x_train, y_train)
alpha_logger.info('Training time cost {}s'.format(time.time() - tic))
alpha_logger.info('best_score = {}, best_round = {}'.format(best_score, best_round))


2019-12-30 05:35:42,418 - ALPHA_MIND - INFO - xgb_model params: 
{'booster': 'gbtree', 'objective': 'reg:linear', 'eval_metric': ['rmse', 'logloss'], 'nthread': 12, 'silent': 1, 'learning_rate': 0.01, 'max_depth': 5, 'eta': 0.03, 'alpha': 0, 'lambda': 0, 'subsample': 0.7, 'colsample_bytree': 0.5, 'min_child_weight': 3, 'seed': 2019}
2019-12-30 05:35:42,418 - ../../../../models/m1_xgb.py[line:95] - INFO: NonCrossValidation。。。。


[0]	train-rmse:0.496794	train-logloss:0.68318
Multiple eval metrics have been passed: 'train-logloss' will be used for early stopping.

Will train until train-logloss hasn't improved in 100 rounds.
[5]	train-rmse:0.472587	train-logloss:0.636186
[10]	train-rmse:0.449568	train-logloss:0.59344
[15]	train-rmse:0.427688	train-logloss:0.55442
[20]	train-rmse:0.406889	train-logloss:0.518671
[25]	train-rmse:0.387113	train-logloss:0.48581
[30]	train-rmse:0.368316	train-logloss:0.45553
[35]	train-rmse:0.350449	train-logloss:0.427555
[40]	train-rmse:0.333461	train-logloss:0.401647
[45]	train-rmse:0.317335	train-logloss:0.377642
[50]	train-rmse:0.301999	train-logloss:0.355317
[55]	train-rmse:0.287422	train-logloss:0.334532
[60]	train-rmse:0.273565	train-logloss:0.315148
[65]	train-rmse:0.260395	train-logloss:0.297045
[70]	train-rmse:0.247878	train-logloss:0.280121
[75]	train-rmse:0.235985	train-logloss:0.264281
[80]	train-rmse:0.224701	train-logloss:0.249459
[85]	train-rmse:0.213975	train-logloss:

2019-12-30 05:35:48,663 - ALPHA_MIND - INFO - Training time cost 6.250922679901123s
2019-12-30 05:35:48,664 - ALPHA_MIND - INFO - best_score = {'best_score': -0.315786}, best_round = 799


In [18]:
# 取调仓日当天的因子数据作为输入.
# total_data_test_excess = train_data[train_data.trade_date == str(ref_date)]
total_data_test_excess = factor_data[factor_data.trade_date == ref_date]

if len(total_data_test_excess) <=0:
    alpha_logger.info('{} HAS NO DATA!!!'.format(ref_date))
    sys.exit()

alpha_logger.info('{0} total_data_test_excess: {1}'.format(ref_date, len(total_data_test_excess)))

# 获取调仓日当天的行业, 风险模型和基准权重数据
industry_matrix = industry_total[industry_total.trade_date == ref_date]
benchmark_weight = benchmark_total[benchmark_total.trade_date == ref_date]
risk_matrix = risk_total[risk_total.trade_date == ref_date]

total_data = pd.merge(industry_matrix, benchmark_weight, on=['code'], how='left').fillna(0.)
total_data = pd.merge(total_data, risk_matrix, on=['code'])
alpha_logger.info('{0} type_of_total_data: {1}'.format(ref_date, type(total_data)))
alpha_logger.info('{0} shape_of_total_data: {1}'.format(ref_date, np.shape(total_data)))
    
total_data_test_excess = pd.merge(total_data, total_data_test_excess, on=['code'])
alpha_logger.info('{0} len_of_total_data_test_excess: {1}'.format(ref_date, len(total_data_test_excess)))

# 股票代码
codes = total_data_test_excess.code.values.tolist()
   
# predict
# alpha_logger.info('total_data_test_excess: \n{}'.format(total_data_test_excess[['weight', 'code', 'industry']]))
x_pred = total_data_test_excess[features]
predict_xgboost = xgb_model.predict(best_model, x_pred)
# alpha_logger.info('predict_xgboost: {}'.format(predict_xgboost))

a = np.shape(predict_xgboost)
predict_xgboost = np.reshape(predict_xgboost, (a[0], -1)).astype(np.float64)
alpha_logger.info('shape_of_predict_xgboost: {}'.format(np.shape(predict_xgboost)))

# 收益率预测结果    
predict_xgboost_df = pd.DataFrame({'xgb_pre': list(predict_xgboost.reshape(-1))})
predict_xgboost_df['trade_date'] = ref_date
predict_xgboost_df['code'] = codes
predict_xgboost_df['code'] = predict_xgboost_df['code'].apply(lambda x: "{:06d}".format(x) + '.XSHG'
                                                              if len(str(x))==6 and str(x)[0] in '6' 
                                                              else "{:06d}".format(x) + '.XSHE')


2019-12-30 05:35:48,717 - ALPHA_MIND - INFO - 2019-12-27 00:00:00 total_data_test_excess: 800
2019-12-30 05:35:48,730 - ALPHA_MIND - INFO - 2019-12-27 00:00:00 type_of_total_data: <class 'pandas.core.frame.DataFrame'>
2019-12-30 05:35:48,730 - ALPHA_MIND - INFO - 2019-12-27 00:00:00 shape_of_total_data: (800, 74)
2019-12-30 05:35:48,739 - ALPHA_MIND - INFO - 2019-12-27 00:00:00 len_of_total_data_test_excess: 800
2019-12-30 05:35:48,755 - ALPHA_MIND - INFO - shape_of_predict_xgboost: (800, 1)


In [19]:
from data.engines.sqlengine import SQLEngine

# 获取当前持仓
record_engine = SQLEngine('sqlite:///./{}/real_tune_record_without_alpha.db'.format(weekly))

try:
    pos_record = record_engine.fetch_record('pos_record')
    previous_pos = pos_record[pos_record['trade_date'] == ref_date_pre]
except Exception as e:
    alpha_logger.info('pos_record Exception:{0}'.format(e))
    previous_pos = pd.DataFrame({'trade_date':[], 'weight':[],'industry':[], 'er':[],'code':[]})

alpha_logger.info('previous_pos_data: {0}, pos_len: {1}'.format(ref_date_pre, len(previous_pos)))

# 股票过滤, 组合优化之前过滤掉(未完成)
## 9:00--9:25之间进行涨跌停股票的实时筛选

# 导入昨持仓并与股票池中所有股票合并, 
if len(previous_pos) <= 0:
    current_position = None
else:
    previous_pos = total_data_test_excess[['code']].merge(previous_pos, on=['code'], how='left',).fillna(0)
    current_position = previous_pos.weight.values
alpha_logger.info('previous_pos:\n {}'.format(previous_pos))

# previous_pos = total_data_test_excess[['code']].merge(previous_pos, on=['code'], how='left').fillna(0)
# current_position = previous_pos.weight.values

# print(current_position.shape)
# print(total_data_test_excess.shape)
# print(previous_pos.shape)
current_position

2019-12-30 05:35:48,786 - ALPHA_MIND - INFO - previous_pos_data: 2019-12-20 00:00:00, pos_len: 0
2019-12-30 05:35:48,787 - ALPHA_MIND - INFO - previous_pos:
 Empty DataFrame
Columns: [weight, industry, er, code, trade_date]
Index: []


In [20]:
# Constraintes settings
industry_names = industry_list(industry_name, industry_level)
constraint_risk = ['EARNYILD', 'LIQUIDTY', 'GROWTH', 'SIZE', 'SIZENL', 'BETA', 'MOMENTUM'] + industry_names
filter_names = industry_names = ['银行','房地产','保险','证券','多元金融']

total_risk_names = constraint_risk + ['benchmark', 'total']

b_type = []
l_val = []
u_val = []

for name in total_risk_names:
    if name == 'benchmark':
        b_type.append(BoundaryType.RELATIVE)
        l_val.append(0.0)
        u_val.append(1.0)
    elif name == 'total':
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-0.0)
        u_val.append(0.0)
    elif name == 'SIZE':
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-1.0)
        u_val.append(1.0)
    elif name == 'SIZENL':
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-1.0)
        u_val.append(1.0)
    elif name in industry_names:
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-0.005)
        u_val.append(0.005)
    else:
        b_type.append(BoundaryType.ABSOLUTE)
        l_val.append(-2.0)
        u_val.append(2.0)
bounds = create_box_bounds(total_risk_names, b_type, l_val, u_val)


In [21]:
benchmark_w = total_data_test_excess.weight.values
alpha_logger.info('type_of_benchmark_w: {}, shape_of_benchmark_w: {}'.format(type(benchmark_w), 
                                                                             np.shape(benchmark_w)))
is_in_benchmark = (benchmark_w > 0.).astype(float).reshape((-1, 1))

# 风险模型数据合并
weight_gap = 0.02
total_risk_exp = np.concatenate([total_data_test_excess[constraint_risk].values.astype(float),
                                 is_in_benchmark,
                                 np.ones_like(is_in_benchmark)],
                                axis=1)

alpha_logger.info('shape_of_total_risk_exp_pre: {}'.format(np.shape(total_risk_exp)))
total_risk_exp = pd.DataFrame(total_risk_exp, columns=total_risk_names)
alpha_logger.info('shape_of_total_risk_exp: {}'.format(np.shape(total_risk_exp)))

constraints = LinearConstraints(bounds, total_risk_exp, benchmark_w)
alpha_logger.info('constraints: {0} in {1}'.format(np.shape(constraints.risk_targets()), ref_date))

lbound = np.maximum(0., benchmark_w - weight_gap)
ubound = weight_gap + benchmark_w
alpha_logger.info('lbound: {0} in {1}'.format(np.shape(lbound), ref_date))
alpha_logger.info('ubound: {0} in {1}'.format(np.shape(ubound), ref_date))

# 组合优化
executor = NaiveExecutor()
current_pos = pd.DataFrame()

target_pos, _ = er_portfolio_analysis(predict_xgboost, 
                                      total_data_test_excess['industry'].values,
                                      None,
                                      constraints,
                                      False,
                                      benchmark_w,
                                      method='risk_neutral',
                                      lbound=lbound,
                                      ubound=ubound,
                                      turn_over_target=0.5,
                                      current_position=current_position)
                  
alpha_logger.info('shape_of_target_pos: {}'.format(np.shape(target_pos)))
alpha_logger.info('len_codes:{}'.format(np.shape(codes)))
target_pos['code'] = codes
# alpha_logger.info('target_pos: \n{}'.format(target_pos))

# 换手率计算
executor.set_current(previous_pos)
turn_over_org, current_pos = executor.execute(target_pos=target_pos)
alpha_logger.info('turn_over_org: {}'.format(turn_over_org))
turn_over = turn_over_org / sum(target_pos.weight.values)
alpha_logger.info('turn_over: {}'.format(turn_over))

# 优化后仓位信息
current_pos['trade_date'] = ref_date
alpha_logger.info('{} is finished'.format(ref_date))


2019-12-30 05:35:48,805 - ALPHA_MIND - INFO - type_of_benchmark_w: <class 'numpy.ndarray'>, shape_of_benchmark_w: (800,)
2019-12-30 05:35:48,807 - ALPHA_MIND - INFO - shape_of_total_risk_exp_pre: (800, 37)
2019-12-30 05:35:48,808 - ALPHA_MIND - INFO - shape_of_total_risk_exp: (800, 37)
2019-12-30 05:35:48,810 - ALPHA_MIND - INFO - constraints: (2, 37) in 2019-12-27 00:00:00
2019-12-30 05:35:48,811 - ALPHA_MIND - INFO - lbound: (800,) in 2019-12-27 00:00:00
2019-12-30 05:35:48,811 - ALPHA_MIND - INFO - ubound: (800,) in 2019-12-27 00:00:00
2019-12-30 05:35:48,828 - ALPHA_MIND - INFO - shape_of_target_pos: (800, 3)
2019-12-30 05:35:48,828 - ALPHA_MIND - INFO - len_codes:(800,)
2019-12-30 05:35:48,830 - ALPHA_MIND - INFO - turn_over_org: 1.0000999408618632
2019-12-30 05:35:48,831 - ALPHA_MIND - INFO - turn_over: 1.0
2019-12-30 05:35:48,833 - ALPHA_MIND - INFO - 2019-12-27 00:00:00 is finished


In [22]:
# 修改code格式
# 取TOP N作为真实的下单股票
real_pos = current_pos.sort_values(by='weight', ascending=False)[:50]
real_pos['weight'] = real_pos['weight'] / real_pos['weight'].sum()
real_pos

,weight,industry,er,code,trade_date
691,0.035786,非银金融,0.009621,601318,2019-12-27
80,0.025687,电子,0.014027,725,2019-12-27
675,0.025067,银行,0.003432,601169,2019-12-27
728,0.024048,银行,0.004135,601818,2019-12-27
651,0.023018,银行,0.004182,601009,2019-12-27
798,0.021568,电子,0.014247,603986,2019-12-27
569,0.021508,非银金融,0.012093,600705,2019-12-27
244,0.021478,电子,0.017636,2456,2019-12-27
266,0.021418,传媒,0.012142,2555,2019-12-27
245,0.021328,有色金属,0.028518,2460,2019-12-27


In [23]:
# 保存记录当前持仓信息, 写入数据库
previous_record = record_engine.fetch_record_meta(Record, ref_date)
if len(previous_record) == 0:
    record_engine.write_data('pos_record', real_pos)
else:
    record_engine.del_historical_data(Record, ref_date)  # 删除同日期的历史数据
    tmp_record = record_engine.fetch_record_meta(Record, ref_date)
    if len(tmp_record) == 0:  # 删除成功
        record_engine.write_data('pos_record', real_pos)
    else:
        print('{} 的数据没有删除: {}'.format(ref_date, len(previous_record)))


In [24]:
# 生成交易记录
## 修改code格式
real_pos['code'] = real_pos['code'].apply(lambda x: "{:06d}".format(x) + '.SH' 
                                          if len(str(x))==6 and str(x)[0] in '6' 
                                          else "{:06d}".format(x) + '.SZ')

real_pos = real_pos.loc[:, ['code', 'weight', 'trade_date']]
real_pos.rename(columns={"code": "证券代码", "weight": "持仓权重", "trade_date": "成分日期"}, inplace=True)
real_pos['交易价格'] = 0

real_pos = real_pos[['证券代码', '持仓权重', '交易价格', '成分日期']].copy()
real_pos.to_csv('./{}/800-base-{}.csv'.format(weekly, end_date), encoding='utf_8_sig', index=False)

In [25]:
previous_record = record_engine.fetch_record_meta(Record, ref_date)
# previous_record = previous_record[previous_record['trade_date'] == ref_date]
previous_record

,trade_date,weight,industry,er,code
0,2019-12-27 00:00:00.000000,0.035786,非银金融,0.009621,601318
1,2019-12-27 00:00:00.000000,0.025687,电子,0.014027,725
2,2019-12-27 00:00:00.000000,0.025067,银行,0.003432,601169
3,2019-12-27 00:00:00.000000,0.024048,银行,0.004135,601818
4,2019-12-27 00:00:00.000000,0.023018,银行,0.004182,601009
5,2019-12-27 00:00:00.000000,0.021568,电子,0.014247,603986
6,2019-12-27 00:00:00.000000,0.021508,非银金融,0.012093,600705
7,2019-12-27 00:00:00.000000,0.021478,电子,0.017636,2456
8,2019-12-27 00:00:00.000000,0.021418,传媒,0.012142,2555
9,2019-12-27 00:00:00.000000,0.021328,有色金属,0.028518,2460
